<a href="https://colab.research.google.com/github/google-research/protein-ligand-binding-free-energy-calculations/blob/dev/colab_tutorial/gromacs_abfe_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2022 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# ABFE Tutorial

## Note

1. If you use a public colab instance, remember to change your runtime type to GPU
2. If you deployed a custom GCE runtime, please connect to a GPU VM instance.

## Summary

1. Pull Gromacs 2022.3 binary compiled specifically for Colab GPU environment and install the software.
2. Install Gromacs Python API on the fly, together with other packages used in the tutorial.
3. Run simulations for benchmarks and show how to use the official GMX
Python API to modify inputs.
4. Visualize the output conformation for the longer simulation.

In [2]:
#@title Install Gromacs and Gromacs Python API
!wget https://storage.googleapis.com/gromacs-bin/gromacs-avx2_256-cuda-11_2.tar.gz -O /tmp/gromacs.tar.gz
!wget https://storage.googleapis.com/gromacs-bin/benchMEM.tpr -O /tmp/benchMEM.tpr
!wget https://storage.googleapis.com/gromacs-bin/hif2a_eq.tpr -O /tmp/hif2a_eq.tpr

!tar zxf /tmp/gromacs.tar.gz
!sudo rm -rf /usr/local/gromacs
!sudo mv gromacs-avx /usr/local/gromacs
!rm /tmp/gromacs.tar.gz

!pip3 install --upgrade pip
!pip3 install setuptools wheel cmake pybind11 py3DMol
!pip3 install git+https://github.com/deGrootLab/pmx.git@abfe_dev
!gmxapi_ROOT=/usr/local/gromacs/ pip3 install --no-cache-dir gmxapi

--2022-10-28 22:05:21--  https://storage.googleapis.com/gromacs-bin/gromacs-avx2_256-cuda-11_2.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23920923 (23M) [application/x-gzip]
Saving to: ‘/tmp/gromacs.tar.gz’

/tmp/gromacs.tar.gz 100%[===================>]  22.81M  --.-KB/s    in 0.1s    

2022-10-28 22:05:21 (165 MB/s) - ‘/tmp/gromacs.tar.gz’ saved [23920923/23920923]

--2022-10-28 22:05:21--  https://storage.googleapis.com/gromacs-bin/benchMEM.tpr
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3461380 (3.3M) [application/octet-stream]
Saving to: ‘/tmp

In [3]:
#@title Set Level for Logging
#@markdown Sets the python logging level. This is useful for debugging.
import logging

log_level = 'DEBUG' #@param ['DEBUG', 'INFO', 'ERROR']

_log_level = {
    'DEBUG': logging.DEBUG,
    'INFO': logging.INFO,
    'ERROR': logging.ERROR
}

logging.basicConfig(level=_log_level[log_level])

In [4]:
#@title [Please Rerun after Restart] Create a Session
#@markdown Please rerun this cell every time when you restart the runtime.
#@markdown A new session ID will be created for a new colab runtime session.
#@markdown Run this cell again will refresh the session ID.

import os
import time

import gmxapi as gmx


# Sets a session ID for this session.
_SESSION_ID = str(int(time.time())) #@param

# Monkey patch this session ID into the gmxapi.operation.ResourceManager.
# Hack. Do not use in the user code.
# This is temporarily added to allow users to start from a new set of working
# directories for all the mdrun launches. GMX API doesn't expose the
# underlying resource manager or context yet.
def operation_id(self):
  return f"{self._base_operation_id}_{_SESSION_ID}"
gmx.operation.ResourceManager.operation_id = property(operation_id)


def get_work_dir(md_output):
  values = [v for v in md_output.values()]
  return values[0].result()


def tail_log(md_output, output_file_base, last_n_lines=6):
  work_dir = get_work_dir(md_output)

  with open(os.path.join(work_dir, output_file_base + '.log'), 'r') as log:
    for line in log.readlines()[-last_n_lines:]:
      print(line)

print("Session ID: ", _SESSION_ID)

INFO:gmxapi:Importing gmxapi.
INFO:gmxapi.operation:Importing gmxapi.operation
DEBUG:gmxapi.operation:Creating runner for signature (*, front: gmxapi.datamodel.NDArray = (), back: gmxapi.datamodel.NDArray = ()) -> gmxapi.datamodel.NDArray.
INFO:gmxapi.commandline:Importing gmxapi.commandline
DEBUG:gmxapi.operation:Creating runner for signature (command: gmxapi.datamodel.NDArray, shell: bool, env: dict, output, stdin: str = '', _exist_ok=True).
DEBUG:gmxapi.operation:Creating runner for signature (filemap: dict) -> list.
INFO:gmxapi.simulation.workflow:Importing gmx.workflow
INFO:gmxapi.simulation.workflow:Using schema version gmxapi_workspec_0_1.
INFO:gmxapi.mdrun:Importing gmxapi.simulation.mdrun
INFO:gmxapi.read_tpr:Importing gmxapi.simulation.read_tpr
INFO:gmxapi.modify_input:Importing gmxapi.simulation.modify_input


Session ID:  1666994793


In [5]:
#@title Run a Short Simulation for Benchmark

# Read a TPR sample and run it.
# The TPR file configures a 20ps simulation.
input_tpr = gmx.read_tpr('/tmp/benchMEM.tpr')

# Can specify computations to GPU.
output_file_base = 'short'
md = gmx.mdrun(input_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})

# Run the simulation
md.run()

# Tail performance
tail_log(md.output, output_file_base)

DEBUG:gmxapi.operation:Updating Sink for source filename: /tmp/benchMEM.tpr.
DEBUG:gmxapi.operation:Source matches sink. No update necessary.
DEBUG:gmxapi.operation:SinkTerminal configured: <class 'gmxapi.operation.SinkTerminal'>
DEBUG:gmxapi.operation:Resolving filename for <SinkTerminal: width=1, InputCollectionDescription([('filename', <class 'str'>)])>.
DEBUG:gmxapi.operation:Input filename:(str) provided by a local constant of type <class 'str'>.
DEBUG:gmxapi.operation:Created data edge <DataEdge: source_collection=DataSourceCollection([('filename', '/tmp/benchMEM.tpr')]), sink_terminal=<SinkTerminal: width=1, InputCollectionDescription([('filename', <class 'str'>)])>> with Sink <SinkTerminal: width=1, InputCollectionDescription([('filename', <class 'str'>)])>
INFO:gmxapi.mdrun:Building mdrun operation from source <gmxapi.simulation.read_tpr.StandardOperationHandle object at 0x7f209599bc10>
INFO:gmxapi.mdrun:mdrun receiving input _simulation_input: ResultDescription(dtype=<class '

               Core t (s)   Wall t (s)        (%)

       Time:      137.896       34.474      400.0

                 (ns/day)    (hour/ns)

Performance:       50.129        0.479

Finished mdrun on rank 0 Fri Oct 28 22:07:16 2022





In [ ]:
#@title Modified Params and Run A Longer Simulation
# Run the simulation for 40 ps.
output_file_base = 'long2'

modified_tpr = gmx.modify_input(
    input=input_tpr, parameters={'nsteps': 20000, 'nstlist': 100})
md = gmx.mdrun(modified_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})
md.run()

# Tail performance
tail_log(md.output, output_file_base)

In [ ]:
#@title What about a binding affinity simulation with HIF2a?
output_file_base = 'hif'

input_tpr = gmx.read_tpr('/tmp/hif2a_eq.tpr')
modified_tpr = gmx.modify_input(
    input=input_tpr, parameters={'nsteps': 20000, 'nstlist': 100})
md = gmx.mdrun(modified_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})
md.run()

# Tail performance
tail_log(md.output, output_file_base)

## Visualization

Visualize the output from the above HIF2A simulation.
We use py3DMol to visualize the output gro file.

In [ ]:
#@title Visualize the output conformation of the HIF2A simulation.
import py3Dmol

view = py3Dmol.view()
work_dir = get_work_dir(md.output)

view.addModel(
    open(os.path.join(work_dir, output_file_base + '.gro'), 'r').read(), 'gro')

view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({}, {'cartoon': {'color': 'spectrum'}})

view.show()

# ABFE Workflow
-----

In [60]:
import numpy as np
import pmx
from pmx.AbsoluteDG import *
import shutil

In [61]:
#@title Copy input files

_WORKDIR = f"/content/pmxrun_{_SESSION_ID}"
os.mkdir(_WORKDIR)

shutil.copytree('/usr/local/lib/python3.7/dist-packages/pmx/abfe_scripts/lysopath/', f'{_WORKDIR}/lysopath/')
shutil.copytree('/usr/local/lib/python3.7/dist-packages/pmx/abfe_scripts/mdppath/', f'{_WORKDIR}/mdppath/')
shutil.copytree('/usr/local/lib/python3.7/dist-packages/pmx/abfe_scripts/struct_top/', f'{_WORKDIR}/struct_top/')

!ls -l $_WORKDIR

total 12
drwxr-sr-x 3 root root 4096 Oct 28 22:06 lysopath
drwxr-sr-x 2 root root 4096 Oct 28 22:06 mdppath
drwxr-sr-x 4 root root 4096 Oct 28 22:06 struct_top


In [63]:
#@title Setup folder structure

# Initialize the free energy environment object. It will store the main 
# parameters for the calculations.
fe = AbsoluteDG(ligList=['lysozyme_benzene'], 
                apoCase='lysozyme_apo', 
                bDSSB=False)

# set the workpath
fe.workPath = f'{_WORKDIR}/lysopath'
# set the path to the molecular dynamics parameter files
fe.mdpPath = f'{_WORKDIR}/mdppath'
# set the number of replicas (several repetitions of calculation are useful to obtain reliable statistics)
fe.replicas = 3
# provide the path to the structures and topologies
fe.structTopPath = f'{_WORKDIR}/struct_top'

# finally, let's prepare the overall free energy calculation directory structure
fe.simTypes = ['em_posre', 'eq_posre', 'eq', 'transitions']

fe.prepareFreeEnergyDir()


---------------------
Summary of the setup:
---------------------

   workpath: /content/pmxrun_1666994793/lysopath
   mdp path: /content/pmxrun_1666994793/mdppath
   # ligands: 1
   ligands:
        lysozyme_benzene
   apo state: lysozyme_apo

---------------------
Directory structure:
---------------------

/content/pmxrun_1666994793/lysopath/
|
|--ligX
|--|--water
|--|--|--stateA
|--|--|--|--run1/2/3
|--|--|--|--|--/em_posre/eq_posre/eq/transitions
|--|--|--stateB
|--|--|--|--run1/2/3
|--|--|--|--|--/em_posre/eq_posre/eq/transitions
|--|--protein
|--|--|--stateA
|--|--|--|--run1/2/3
|--|--|--|--|--/em_posre/eq_posre/eq/transitions
|--|--|--stateB
|--|--|--|--run1/2/3
|--|--|--|--|--/em_posre/eq_posre/eq/transitions
|--|--strTopFolder
|--lig..

DONE


In [64]:
#@title Assemble simulation systems

# assemble the systems 
fe.assemble_systems()

# box/water/ions
fe.boxWaterIons()

----------------------
Assembling the systems
----------------------
Order: ligand-protein-other-water
--- Assembling structures: lysozyme_benzene ---
--- Assembling topologies: lysozyme_benzene ---
----------------
Box, water, ions
----------------
TODO


In [72]:
!ls $_WORKDIR/lysopath/lysozyme_benzene/protein/stateA/run1/

In [ ]:
#@title Prepare simulations

# em_posre
#fe.prepare_simulation( simType='em_posre' )
fe.JOBsimcpu = 4
#fe.JOBmodules = ['shared','owl/intel-mpi-default','cuda91']
#fe.JOBsource = ['/etc/profile.d/modules.sh']
fe.JOBqueue = 'SLURM'
fe.JOBmodules = ['owl/intel-mpi-default','cuda/11.0','hwloc']
fe.JOBgmx = '/usr/local/gromacs/2021/2021.2-impi2017-fftw337-gcc93-cuda11.1/bin/mdrun_threads_AVX_256'
#fe.prepare_jobscripts( simType='em_posre' )

# eq_posre
#fe.prepare_simulation( simType='eq_posre', prevSim='em_posre' )
fe.JOBsimcpu = 8
#fe.prepare_jobscripts( simType='eq_posre' )

# eq
#fe.prepare_simulation( simType='eq', prevSim='eq_posre' )
fe.JOBsimcpu = 20
#fe.prepare_jobscripts( simType='eq' )

# transitions
fe.equilTime = 1080.0 # ps to discard as equilibration
#fe.bGenTiTpr = True
fe.JOBgmx = '/usr/local/gromacs/2021/2021.2-impi2017-fftw337-gcc93-cuda11.1/bin/mdrun_threads_AVX_256'
#fe.prepare_simulation( simType='transitions' )

# jobscript for transitions
fe.JOBgmx = '/usr/local/gromacs/2021/2021.2-impi2017-fftw337-gcc93-cuda11.1/bin/mdrun_threads_AVX_256'
fe.JOBqueue = 'SLURM'
fe.JOBmodules = ['owl/intel-mpi-default','cuda/11.0','hwloc']
fe.JOBsimcpu = 4
#fe.prepare_jobscripts( simType='transitions' )

# analysis
#fe.run_analysis(ligs=['lysozyme_benzene'] )
#fe.analysis_summary()

This is the structure for each calculation.

```
ligand1/
  water/                # ligand in water simulations
    stateA/             # coupled ligand (i.e. solvated)
      run1/             # only 1 run as example, but we'll have >1
        em/             # energy minimization
        eq_posre/       # equilibration with position restraints
        eq/             # equilibrium simulation
        transitions/    # non-equilibrium transitions
          frame1/       # individual non-eq runs starting
          frame2/
          ...
          frame100/    
    stateB              # decoupled ligand (i.e. in vacuum)
      run1/
        em/
        eq_posre/
        eq/
        transitions/
  protein/              # ligand in protein simulations
    stateA/             # coupled ligand (i.e. protein-ligand complex)
      run1/
        em/
        eq_posre/
        eq/
        transitions/
    stateB/             # decoupled ligand (i.e. apo protein + restr ligand)
      run1/
        em/
        eq_posre/
        eq/
        transitions/
```

### Setup protein-ligand restraints
This step identifies the set of restraints to be used for the `protein/stateB` simulations.

--> Matteo: we might be using a new approach for hanlding the restraints in `stateA`. I need to double check with Vytas how this is done in practice with pmx.

## 2. Prepare and run equilibrium simulations
This will prepare and run all simulations in folders `em/`, `eq_posre`, and `eq`.

## 3. Prepare and run non-equilibrium simulations
This extracts N frames from the equilibrium trajectories in the `eq/` folders, puts them in the `frame*/` folders and runs a non-equilibrium simulation from each frame. Each non-equilibrium simulation interpolates between `stateA` and `stateB` (i.e., between the "coupled" and "decoupled" ligand).

## 4. Estimate binding free energy
The analysis will take in .xvg files present in each `frame*/` folder and return the estimated free energy differences.